In [ ]:
import RNA
import csv

def calculate_pair_probabilities(sequence, output_csv):
    """
    Calculate base pair probabilities and unpaired probabilities for an RNA sequence and save to CSV.
    
    Args:
        sequence (str): RNA sequence (e.g., "GGGAAACCC")
        output_csv (str): Path to output CSV file
    """
    # Create fold compound object
    fc = RNA.fold_compound(sequence)
    
    # Calculate partition function and pair probabilities
    (ss, mfe) = fc.mfe()
    fc.pf()
    pair_probs = fc.bpp()
    
    # Calculate unpaired probabilities (1 - sum of all pairing probabilities for each position)
    unpaired_probs = [1.0 for _ in range(len(sequence))]
    for i in range(len(sequence)):
        for j in range(i+1, len(sequence)):
            prob = pair_probs[i+1][j+1]  # +1 because ViennaRNA uses 1-based indexing
            if prob > 0.01:  # Only consider probabilities > 1%
                unpaired_probs[i] -= prob
                unpaired_probs[j] -= prob
    
    # Write probabilities to CSV
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(['i', 'j', 'probability', 'type'])
        
        # Write all non-zero pair probabilities (i < j)
        for i in range(len(sequence)):
            for j in range(i+1, len(sequence)):
                prob = pair_probs[i+1][j+1]
                if prob > 0.01:  # Only store probabilities > 1%
                    writer.writerow([i+1, j+1, prob, 'pair'])  # Using 1-based indices in output
        
        # Write unpaired probabilities
        for i in range(len(sequence)):
            writer.writerow([i+1, '-', unpaired_probs[i], 'unpaired'])

    print(f"Pair and unpaired probabilities saved to {output_csv}")

# Example usage
sequence = "atggtgagcaagggcgaagaagataacatggccatcatcaaggagttcatgcgcttcaaggttcacatggagggctccgtgaacggccacgagttcgagatcgagggcgagggcgagggccgcccctacgagggcacccagaccgccaagctgaaggtgaccaagggtggccccctgcccttcgcctgggacatcctgtcccctcagttcatgtacggctccaaggcctacgtgaagcaccccgccgacatccccgactacttgaagctgtccttccccgagggcttcaagtgggagcgcgtgatgaacttcgaggacggcggcgtggtgaccgtgacccaggactcctccctgcaagacggcgagttcatctacaaggtgaagctgcgcggcaccaacttcccctccgacggccccgtaatgcagaagaagactatgggctgggaggcctcctccgagcggatgtaccccgaggacggcgcgctgaagggcgagatcaagcagaggctgaagctgaaggacggcggccactacgacgctgaggtcaagaccacctacaaggccaagaagcccgtgcaactgcccggcgcgtacaacgtcaacatcaagttggacatcacctcccacaacgaggactacaccatcgtggaacagtacgaacgcgccgagggccgccactccaccggcggcatggacgagctgtacaagtaa"  # Your RNA sequence here
output_file = "viennarna_probabilities.csv"
calculate_pair_probabilities(sequence, output_file)